## Vector Search on Azure Cache for Redis

### Set environment variables

In [ ]:
## TODO

#### Simple vector search

In [ ]:
## TODO

### Cross column vector similarity search

In [ ]:
## TODO

### Hybrid search

In [ ]:
## TODO

#### Document search example

In [ ]:
## TODO

#### Image search example

In [ ]:
## TODO